In [1]:
from utils.russian_superglue_models import SpanClassificationModel
from transformers import AutoModel, PreTrainedModel

In [18]:
from utils.dataset_configs import (
    TASK_NUM_CLASSES,
    TASK_TO_CONFIG,
    TASK_TO_NAME,
    TASK_TYPES,
)
from utils.russian_superglue_models import (
    BertForEntityChoice,
    RobertaForEntityChoice,
    SpanClassificationModel,
    EntityChoiceModel
)


In [14]:
from dataclasses import dataclass 
from typing import Dict
from transformers import BertConfig, RobertaConfig, AutoModelForSequenceClassification, BertTokenizer, RobertaTokenizer

@dataclass
class ModelData:
    config: object
    tokenizer: object
    task_types: Dict[str, object]


MODEL_CLASSES: Dict[str, ModelData] = {
    "bert": ModelData(
        config=BertConfig,
        tokenizer=BertTokenizer,
        task_types={
            "classification": BertForSequenceClassification,
            "entity_choice": BertForEntityChoice,
            "span_classification": SpanClassificationModel,
        },
    ),
    "roberta": ModelData(
        config=RobertaConfig,
        tokenizer=RobertaTokenizer,
        task_types={
            "classification": RobertaForSequenceClassification,
            "entity_choice": RobertaForEntityChoice,
            "span_classification": SpanClassificationModel,
        },
    ),
}

In [15]:
def get_model(args) -> PreTrainedModel:
    """
    Returns a pre-trained model for a given task.

    Args:
        args: An object that contains the following fields:
            - model_name: A string that represents the name of the pre-trained model.
            - task_name: A string that represents the name of the task.

    Returns:
        An instance of a pre-trained model for the given task.

    Raises:
        ValueError: If the model or task name is not found in the dictionaries.
    """
    model_data = MODEL_CLASSES.get(args.model_name)
    if not model_data:
        raise ValueError(f"Unknown model name: {args.model_name}")
    model_type = model_data.task_types.get(TASK_TYPES[args.task_name])
    if not model_type:
        raise ValueError(f"Unknown task name: {args.task_name}")
    num_classes = TASK_NUM_CLASSES.get(args.task_name, 2)
    if TASK_TYPES[args.task_name] == 'span_classification':
        return SpanClassificationModel(
            backbone=AutoModel.from_pretrained(args.model_name_or_path),
            num_labels=num_classes,
        )
    elif TASK_TYPES[args.task_name] == 'entity_choice':
        return EntityChoiceModel(
            backbone=AutoModel.from_pretrained(args.model_name_or_path)
        )
    else:
        return AutoModelForSequenceClassification.from_pretrained(
            args.model_name_or_path, num_labels=num_classes
        )

In [16]:
class Args:
    def __init__(self, task_name, model_name, model_name_or_path):
        self.task_name = task_name
        self.model_name = model_name
        self.model_name_or_path = model_name_or_path

In [19]:
for task in TASK_TO_NAME.keys():
    args = Args(task, 'bert', 'bert-base-uncased')
    get_model(args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

TypeError: EntityChoiceModel.__init__() got an unexpected keyword argument 'num_labels'

In [7]:
args = Args('russe', 'bert')

In [8]:
get_model(args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SpanClassificationModel(
  (backbone): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [2]:
import os
from datasets import Dataset, DatasetDict
from utils.dataset_configs import TASK_TO_NAME, load_data

In [82]:
dataset = load_data('rwsd')

Using custom data configuration default-6737211bbf7a4738
/mnt/storage/moskovskiy/workspace/anaconda3/envs/tc/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Using custom data configuration default-0e43f68e0446fee4
Found cached dataset json (/home/moskovskiy/.cache/huggingface/datasets/json/default-0e43f68e0446fee4/0.0.0)


Dataset json downloaded and prepared to /home/moskovskiy/.cache/huggingface/datasets/json/default-6737211bbf7a4738/0.0.0. Subsequent calls will reuse this data.


In [4]:
from utils.dataset_configs import TASK_TO_CONFIG
config = TASK_TO_CONFIG['muserc'](dataset)

In [4]:
from transformers import BertTokenizerFast
from functools import partial
tokenizer = BertTokenizerFast.from_pretrained('cointegrated/rubert-tiny2')
processed_dataset = dataset.map(
    partial(
        config.process_data, tokenizer=tokenizer, max_length=512
    ),
    num_proc=32,
    keep_in_memory=True,
    batched=True,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

KeyError: 'question'

In [10]:
print(max([len(x["text"]) for x in dataset["train"]]))

245


In [43]:
dataset["train"][0]

{'idx': 0,
 'target': {'span1_text': 'Члены городского совета',
  'span2_text': 'они опасались',
  'span1_index': 0,
  'span2_index': 10},
 'label': True,
 'text': 'Члены городского совета отказали организаторам митинга в разрешении, потому что они опасались насилия.'}

In [76]:
from utils.dataset_configs import DatasetConfig
from transformers import PreTrainedTokenizer, EvalPrediction
from typing import List 

def find_sub_list(sublist: List[int], main_list: List[int]):
    sublist_length=len(sublist)
    for idx in (index for index, element in enumerate(main_list) if element == sublist[0]):
        if main_list[idx: idx + sublist_length] == sublist:
            start = idx
            end = idx + sublist_length 
    print(start, end)
    return start, end  

class RWSDConfig(DatasetConfig):

    best_metric: str = "accuracy"
    num_classes: int = 2
    
    @staticmethod
    def process_data(examples, tokenizer: PreTrainedTokenizer, max_length: int):
        # print(examples)
        result = tokenizer(
            examples["text"],
            truncation="longest_first",
            return_token_type_ids=True,
            max_length=245,
        )
        # print(result)

        e1_masks, e2_masks = [], [] 
        
        for i, sample in enumerate(examples["target"]):
            e1_mask = np.zeros_like(result["input_ids"], dtype=int)
            e2_mask = np.zeros_like(result["input_ids"], dtype=int)

            e1_span = tokenizer(
                sample["span1_text"],
                add_special_tokens=False,
                return_attention_mask=False,
                return_token_type_ids=False,
            )["input_ids"]
            e2_span = tokenizer(
                sample["span2_text"],
                add_special_tokens=False,
                return_attention_mask=False,
                return_token_type_ids=False,
            )["input_ids"]
            for mask, span in zip((e1_mask, e2_mask), (e1_span, e2_span)):
                start, end = find_sub_list(span, result["input_ids"][i])
                mask[start: end] = 1

            e1_masks.append(e1_mask)
            e2_masks.append(e2_mask)

        result["e1_mask"] = e1_masks
        result["e2_mask"] = e2_masks

        if isinstance(examples["label"], list):
            result["labels"] = [int(x) for x in examples["label"]]
        else:
            result["labels"] = int(examples["label"])
        
        return result  

    def compute_metrics(self, predictions: EvalPrediction, split: str, **kwargs):
        preds = np.argmax(predictions.predictions, axis=1)
        return {
            "accuracy": accuracy_score(
                y_true=predictions.label_ids.astype(np.float32), y_pred=preds
            )
        }

In [77]:
config = RWSDConfig(dataset)

In [79]:
# from transformers import BertTokenizerFast
# from functools import partial
# tokenizer = BertTokenizerFast.from_pretrained('cointegrated/rubert-tiny2')
# processed_dataset = dataset.map(
#     partial(
#         config.process_data, tokenizer=tokenizer, max_length=255
#     ),
#     num_proc=32,
#     keep_in_memory=True,
#     batched=True,
# )

In [83]:
for sample in dataset["train"]:
    result = tokenizer(
        sample["text"],
        truncation="longest_first",
        return_token_type_ids=True,
        max_length=245,
        )

    e1_mask, e2_mask = tokenizer(sample["target"]["span1_text"], add_special_tokens=False)["input_ids"], \
        tokenizer(sample["target"]["span2_text"], add_special_tokens=False)["input_ids"] 
    print(find_sub_list(e1_mask, result["input_ids"]), end='\r')
    print(find_sub_list(e2_mask, result["input_ids"]), end='\r')



1 4
12 15)
5 7, 15)
12 15)
1 4, 15)
12 15)
5 7, 15)
12 15)
1 2, 15)
10 13)
6 7, 13)
10 13)
1 2, 13)
10 13)
6 7, 13)
10 13)
1 2, 13)
9 112)
3 5 11)
9 115)
1 2 11)
9 112)
3 5 11)
9 115)
1 2 11)
9 132)
3 5 13)
9 135)
1 2 13)
9 122)
3 5 12)
9 125)
1 2 12)
8 102)
3 5 10)
8 105)
1 2 10)
9 112)
3 5 11)
9 115)
1 4 11)
10 12)
6 7, 12)
10 12)
1 4, 12)
10 12)
6 7, 12)
10 12)
1 2, 12)
16 18)
12 1318)
16 1813)
1 2, 18)
16 18)
12 1318)
16 1813)
1 2, 18)
10 12)
6 7, 12)
10 12)
1 2, 12)
10 12)
6 7, 12)
10 12)
3 4, 12)
9 114)
5 6 11)
9 116)
3 4 11)
9 114)
5 6 11)
9 116)
1 2 11)
14 16)
9 10 16)
14 160)
1 2, 16)
14 16)
9 10 16)
14 160)
1 2, 16)
9 112)
5 7 11)
9 117)
1 2 11)
9 112)
5 7 11)
9 117)
10 111)
18 1911)
13 1519)
18 1915)
10 1119)
18 1911)
13 1519)
18 1915)
1 5, 19)
12 13)
8 9, 13)
12 13)
1 5, 13)
12 13)
8 9, 13)
12 13)
1 3, 13)
13 15)
7 10 15)
13 150)
1 3, 15)
13 15)
7 10 15)
13 150)
1 2, 15)
13 15)
8 10 15)
13 150)
1 2, 15)
13 15)
8 10 15)
13 150)
1 3, 15)
11 14)
7 8, 14)
11 14)
1 3, 14)
11 14)

UnboundLocalError: local variable 'start' referenced before assignment

In [85]:
sample

{'idx': 64,
 'target': {'span1_text': 'Фрэнк',
  'span2_text': 'он купил',
  'span1_index': 0,
  'span2_index': 10},
 'label': True,
 'text': 'Фрэнк подал в суд на Тома потому, что тостер, который он у него купил не работал.'}

In [11]:
from typing import List
def find_sub_list(sublist: List[str], main_list: List[str]):
    sublist_length = len(sublist)
    for idx, word in enumerate(main_list):
        if word == sublist[0] and main_list[idx:idx+sublist_length] == sublist:
            start = idx
            end = idx + sublist_length
            return start, end
    return None, None


In [31]:
from transformers import PreTrainedTokenizer, BertTokenizer
import numpy as np

def process_data(examples, tokenizer: PreTrainedTokenizer, max_length: int):
    result = tokenizer(
        examples["text"],
        truncation="longest_first",
        return_token_type_ids=True,
        max_length=max_length,
    )



    e1_mask = np.zeros_like(result["input_ids"], dtype=int)
    e2_mask = np.zeros_like(result["input_ids"], dtype=int)

    e1_span = sample["target"]["span1_text"]
    e2_span = sample["target"]["span2_text"]

    # Find the start and end indices of the spans in the input text
    e1_start, e1_end = find_sub_list(e1_span.split(), examples["text"].split())
    e2_start, e2_end = find_sub_list(e2_span.split(), examples["text"].split())

        # If the spans are not found, just use the indices of the span words
    if e1_start is None:
        e1_start, e1_end = [i for i, x in enumerate(examples["text"].split()) if x in e1_span.split()], [i+1 for i, x in enumerate(examples["text"].split()) if x in e1_span.split()]
    if e2_start is None:
        e2_start, e2_end = [i for i, x in enumerate(examples["text"].split()) if x in e2_span.split()], [i+1 for i, x in enumerate(examples["text"].split()) if x in e2_span.split()]
    print(e1_start, e1_end)
    print(e2_start, e2_end)
    # Set the corresponding mask values to 1 for each span
    e1_mask[e1_start:e1_end] = 1
    e2_mask[e2_start:e2_end] = 1


    result["e1_mask"] = e1_mask
    result["e2_mask"] = e2_mask

    if isinstance(examples["label"], list):
        result["labels"] = [int(x) for x in examples["label"]]
    else:
        result["labels"] = int(examples["label"])
    
    return result

In [32]:
tokenizer = BertTokenizerFast.from_pretrained('cointegrated/rubert-tiny2')

In [33]:
sample = {'idx': 64,
 'target': {'span1_text': 'Фрэнк',
  'span2_text': 'он купил',
  'span1_index': 0,
  'span2_index': 10},
 'label': True,
 'text': 'Фрэнк подал в суд на Тома потому, что тостер, который он у него купил не работал.'}

In [34]:
process_data(sample, tokenizer, 255)

0 1
[10, 13] [11, 14]


TypeError: slice indices must be integers or None or have an __index__ method